# `Luigi`

<img src='img/luigi.png'>


[**Документация Luigi**](https://luigi.readthedocs.io/en/stable/running_luigi.html)

[**GitHub Luigi**](https://github.com/spotify/luigi)

```python 
pip install luigi 
```

___
## <center> <a id=1 >Создание задачи</a>

- $\boxed{1}$ `luigi.Task` : Наследуется класс `Task`

- $\boxed{2}$ def `run`(self) : Исполняемый код помещается в метод `run`
- $\boxed{3}$ `python -m` $\boxed{название \ файла}$ $\boxed{название \ задачи}$ `--local-scheduler`: Запускается пайплайн через консоль



In [2]:
import luigi

class MyTask(luigi.Task):
    def run(self):
        print('Some txt')


if __name__ == '__main__':
    luigi.run()


___
## <center> <a id=2 >Параметры задачи</a>

Параметр позволяет использовать входящие данные
- `IntParameter`
- `FloatParameter`
- `DateParameter`
- `DateIntervalParameter`

В командной строке парметр задается как аргумент с префиксом `--`

`python -m` $\boxed{название \ файла}$ $\boxed{название \ задачи}$  `--`x 123 `--`y 456 `--local-scheduler`


In [ ]:
import luigi

class MyTask(luigi.Task):
    x = luigi.IntParameter()
    y = luigi.IntParameter(default=45)

    def run(self):
        print(self.x + self.y)

if __name__ == '__main__':
    luigi.run()

___
## <center> <a id=3 >Проверка наличия файла</a>

- $\boxed{1}$ `luigi.Task` : Наследуется класс `Task`

- $\boxed{2}$ def `output`(self) : Метод в котором указыватся файл для проверки на наличие (Задача выполнена, если файл существует)

    - - $\boxed{3}$ `LocalTarget`: Определяет объект в виде файла на `локальной` машине


In [ ]:
import luigi

class MyTask(luigi.Task):
    filename = "file_to_check.txt"

    def run(self):                          #  Записываем текст в файл
        with open(self.filename, 'w') as f:
            f.write("Some text")

    def output(self):                       #  Проверяем на наличие
        return luigi.LocalTarget(self.filename)

if __name__ == '__main__':
    luigi.run()

___
## <center> <a id=3 >Зависимости между задачами</a>

- $\boxed{1}$ `luigi.Task` : Наследуется класс `Task` для каждой задачи

- $\boxed{2}$ def `requires`(self) : В зависимой задаче определяется метод `requires`, в котором указывается задачи (только класс Task), которые должны быть выполнены перед зависимой

- $\boxed{3}$ Запускается :

    - Первая задача:
        - Выполняется только первая задача
    - Вторая задача:
        - Проверяется выполнение первой задачи:

            - Если первая уже выполнена, то выполняется вторая
            - Если первая не выполнена, то выполняется первая, а за ней выполняется вторая



In [ ]:
import luigi

class FirstTask(luigi.Task):
    filename = 'first_file.txt'

    def run(self):
        with open(self.filename, 'w') as f:
            f.write('first text')

    def output(self):
        return luigi.LocalTarget(self.filename)

class SecondTask(luigi.Task):
    filename = 'second_file.txt'

    def run(self):
        with open(self.filename, 'w') as f:
            f.write('second text')

    def requires(self):
        return FirstTask()

    def output(self):
        return luigi.LocalTarget(self.filename)

if __name__ == '__main__':
    luigi.run()

___
## <center> <a id=4 >Динамические зависимости</a>

Выполнение задачи MyTask будет приостановлено, пока не будет выполнена задача OtherTask. При этом объект other_target будет объектом класса Target.

In [ ]:
import luigi

class MyTask(luigi.Task):
    other_target = yield OtherTask()

    f = other_target.open('r')

___
## <center> <a id=5 >Планировщик</a>

- Запуск планировщика : luigid --background --pidfile <PATH_TO_PIDFILE> --logdir <PATH_TO_LOGDIR> --state-path <PATH_TO_STATEFILE>
    - <PATH_TO_PIDFILE>, <PATH_TO_LOGDIR>, <PATH_TO_STATEFILE> : пути до технических файлов Luigi

